In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cirq
from cirq.contrib.svg import SVGCircuit
import random as rd
from sympy import *
import tensorflow as tf
import tensorflow_quantum as tfq
import math
import re
import itertools
from numpy import linalg as LA
from scipy.stats import poisson
import time
from random import choices

2023-03-20 20:28:06.176089: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-20 20:28:06.176104: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-20 20:28:06.785819: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-20 20:28:06.785834: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-20 20:28:06.785844: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (rafael-ThinkPad): /proc/driver/nvidia/version does not exist
2023-03-20 20:28:06.785984: I tensorflow/core/platform/cpu_fe

In [2]:
def generate_binary_strings(bit_count):
    binary_strings = []
    def genbin(n, bs=''):
        if len(bs) == n:
            binary_strings.append(bs)
        else:
            genbin(n, bs + '0')
            genbin(n, bs + '1')

    genbin(bit_count)
    return binary_strings

In [3]:
r_by_k = {2 : 1, 3: 6.43, 4: 20.43, 5 : 45.7, 6: 70.21, 8: 176.54, 10: 708.92, 16: 45425.2}

def generate_instance(k: int, n: int) -> np.ndarray:
    #generate an instance of random k-SAT with n variables in the satisfiability threshold
    if not (r := r_by_k.get(k)):
        raise ValueError(f"k must be in {list(r_by_k)} (got {k})")
    m = poisson(r*n).rvs()
    #return np.random.choice(all_vars, size=(m, k))
    all_variables = []
    all_signs = []
    for i in range(m):
        #all_signs.append([rd.choice(l) for i in range(k)])
        all_variables.append(choices(all_vars, k = k))

    all_variables = np.array(all_variables)
    #all_signs = np.array(all_signs)
    return all_variables

In [4]:
def dimacs_writer(dimacs_filename, cnf_array):
    #writes the dimacs file with the CNF
    cnf = cnf_array
    cnf_length = len(cnf)
    n_sat = len(cnf[0])
    var_num = np.max(cnf) 
    with open(dimacs_filename, "w") as f:

        f.write('c DIMACS file CNF '+str(n_sat)+'-SAT \n')
        f.write("p cnf {} {}\n".format(var_num, cnf_length))
        
        for i, clause in enumerate(cnf):
            line = clause.tolist()
            if i == cnf_length - 1:
                s = ' '.join(str(x) for x in line)+' 0'
                f.write(s)
            else: 
                s = ' '.join(str(x) for x in line)+' 0 \n'
                f.write(s)

In [5]:
class Verifier():
    #verifier from Qiskit page, takes a bit string and checks if cnf is satisfied
    def __init__(self, dimacs_file):
        with open(dimacs_file, 'r') as f:
            self.dimacs = f.read()

    def is_correct(self, guess):
        guess = [bool(int(x)) for x in guess][::-1]
        for line in self.dimacs.split('\n'):
            line = line.strip(' 0')
            clause_eval = False
            for literal in line.split(' '):
                if literal in ['p', 'c']:
                    # line is not a clause
                    clause_eval = True
                    break
                if '-' in literal:
                    literal = literal.strip('-')
                    lit_eval = not guess[int(literal)-1]
                else:
                    lit_eval = guess[int(literal)-1]
                clause_eval |= lit_eval
            if clause_eval is False:
                return False
        return True

In [6]:
def mixing_circuit(circuit, qubits, par):
    for i in range(len(qubits)):
        circuit.append(cirq.rx(par).on(qubits[i]))
    return circuit

In [7]:
def ham_layer(diagonal, circuit, qubits, par):
    
    l = cirq.DiagonalGate(diagonal)._decompose_(qubits)
    l.pop(0)
    for j, gate in enumerate(l):
        #print(gate)
        if j % 2 == 0:
            dictn = gate._json_dict_()
            my_string = str(dictn['gate'])
            my_other_string = str(dictn['qubits'])
            number_p = re.findall("\d+\.\d+", my_string)
            res_p = [eval(i) for i in number_p]
            
            if '-' in my_string:
                sign = -1
            else:
                sign = 1
            
            number_q = re.findall(r'\d+', my_other_string)
            res_q = [eval(i) for i in number_q]
            rzgate = cirq.rz(sign*par*res_p[0]*np.pi).on(qubits[res_q[1]])
            circuit.append(rzgate)
        else:
            circuit.append(gate)

In [8]:
class MyLRSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

    def __init__(self, initial_learning_rate):
        self.initial_learning_rate = initial_learning_rate

    def __call__(self, step):
        return self.initial_learning_rate / (step+1)

In [9]:
k = 4 #length of clauses
n_var = 6 #number of variables
p = 6 #number of layers. When in doubt, stay on the lower side
nqubits = n_var #number of qubits in the circuit

all_vars = [i for i in range(-n_var,n_var+1)]
all_vars = [i for i in all_vars if i != 0]
binary_strings = generate_binary_strings(nqubits)
times_list = []

for _ in range(10):
        
    valid_keys = []
    dimacs_filename = "random_cnf_BM.dimacs" 

    while not valid_keys:
        #only accepts satisfiable CNFs
        inst = generate_instance(k, n_var)
        dimacs_writer(dimacs_filename, inst)
        v = Verifier('random_cnf_BM.dimacs')

        for key in binary_strings:
            if v.is_correct(key) == True:
                print(key) 
                valid_keys.append(key)

    with open('random_cnf_BM.dimacs', 'r') as f:
        dimacs = f.read()
    #print(dimacs)
    unsat_list = []

    for key in binary_strings:
        guess = [bool(int(x)) for x in key][::-1]

        clause_eval_list = []
        counter = 0
        for j, line in enumerate(dimacs.split('\n')):

            line = line.strip(' 0')
            clause_eval = False

            for literal in line.split(' '):
                if literal in ['p', 'c']:
                    #line is not a clause
                    clause_eval = True
                    break
                if '-' in literal:
                    literal = literal.strip('-')
                    lit_eval = not guess[int(literal)-1]
                else:
                    lit_eval = guess[int(literal)-1]
                clause_eval |= lit_eval
            #print(clause_eval)
            if j > 1:
                counter += 1
                clause_eval_list.append(clause_eval)
        unsat_clauses = counter - sum(clause_eval_list)
        unsat_list.append(unsat_clauses)

    diagonal = unsat_list

    qubits = []

    for i in range(nqubits):
        qubits.append(cirq.GridQubit(0,i))        

    qubits = list(qubits) #don't know why
    all_qubits = [i for i in range(nqubits)]

    def my_gate(c, index):
        g = c * cirq.Z.on(qubits[index]) + cirq.I.on(qubits[index])
        return g

    x = [1, -1]
    combinations = [p for p in itertools.product(x, repeat=nqubits)]

    ops_list = []
    for j, combination in enumerate(combinations):
        ops_list.append((diagonal[j]/2**nqubits)*math.prod([my_gate(combination[i], i) for i in range(nqubits)]))

    cost = np.sum(ops_list)

    qaoa_circuit = cirq.Circuit()
    num_param = 2 * p 
    parameters = symbols("q0:%d" % num_param)

    #setting up the layers
    for i in range(p):
        ham_layer(diagonal, qaoa_circuit, qubits, parameters[2 * i])
        mixing_circuit(qaoa_circuit, qubits, parameters[2 * i + 1])

    initial = cirq.Circuit()

    for qubit in qubits:
        initial.append(cirq.H(qubit)) #applying Hadamard to all qubits before running circuit

    #setting up the model
    lr = 1e-1

    inputs = tfq.convert_to_tensor([initial])
    ins = tf.keras.layers.Input(shape = (), dtype = tf.dtypes.string)
    outs = tfq.layers.PQC(qaoa_circuit, cost)(ins)
    ksat = tf.keras.models.Model(inputs = ins, outputs = outs)
    opt = tf.keras.optimizers.Adam(learning_rate=MyLRSchedule(lr))
    ksat.trainable_variables[0].assign([0. for i in range(2*p)]) #initializing angles with some small noise

    cost_m = cost.matrix()
    gs_energy = np.real(min(LA.eig(cost_m)[0]))

    losses = []
    error = 1e2*rd.random()
    tol = 1e-1

    start = time.time()

    while _ < 1e6:
        previous_error = error   
        with tf.GradientTape() as tape:
            error = ksat(inputs)

        grads = tape.gradient(error, ksat.trainable_variables)
        opt.apply_gradients(zip(grads, ksat.trainable_variables))
        error = error.numpy()[0,0]
        losses.append(error)

        print('absolute value of (ground state energy - error) is ' + str(abs(gs_energy - error)), end = '\r')

        if abs(error - previous_error) < 1e-6:
            #print('\n got stuck!')
            break

    end = time.time()
    time_training = end - start
    print("Time consumed in training: ", end - start)
    params = ksat.trainable_variables

    start = time.time()

    sample_circuit = tfq.layers.AddCircuit()(inputs, append=qaoa_circuit)
    output = tfq.layers.Sample()(sample_circuit, symbol_names=parameters, symbol_values=params, repetitions = 2048)

    end = time.time()
    time_sampling = end - start

    times_list.append((time_training, time_sampling))

    print("Time consumed in sampling: ", end - start)

001110
Time consumed in training:  158.8470904827118r) is 1.4531704187393188
Time consumed in sampling:  0.17647576332092285
011101
Time consumed in training:  178.58337235450745) is 0.6702287197113037
Time consumed in sampling:  0.2005157470703125
011011
Time consumed in training:  142.42836904525757) is 1.0685062408447266
Time consumed in sampling:  0.12355351448059082
000010
Time consumed in training:  157.20101189613342) is 1.8307896852493286
Time consumed in sampling:  0.22139286994934082
111110
Time consumed in training:  251.41383409500122) is 1.5015695095062256
Time consumed in sampling:  0.12434124946594238
001101
Time consumed in training:  243.05299615859985) is 1.1702055931091309
Time consumed in sampling:  0.12538647651672363
110011
111000
Time consumed in training:  175.81706714630127) is 0.8358893990516663
Time consumed in sampling:  0.13589835166931152
100000
Time consumed in training:  255.93854188919067) is 1.5930780172348022
Time consumed in sampling:  0.124878883361

In [10]:
import pandas as pd
df = pd.DataFrame (times_list, columns = ['training', 'sampling'])
print (df)

     training  sampling
0  158.847090  0.176476
1  178.583372  0.200516
2  142.428369  0.123554
3  157.201012  0.221393
4  251.413834  0.124341
5  243.052996  0.125386
6  175.817067  0.135898
7  255.938542  0.124879
8  214.859555  0.220454
9  141.024681  0.201596


In [11]:
df.to_csv('times_k_'+str(k)+'_vars_'+str(n_var)+'_p_'+str(p)+'.csv', index= False)

In [12]:
k, p

(4, 6)